In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. KNN을 이용하여 이미지 분류

### 1) Pre-processing of data

In [3]:
import numpy as np
from skimage.transform import resize
from skimage.io import imread
import matplotlib.pyplot as plt
import os

from tqdm import tqdm
import pickle

# 병렬연산
from concurrent.futures import ThreadPoolExecutor

import os
import warnings
warnings.filterwarnings(action='ignore')

import cv2
from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split

In [4]:
### 사용자 정의 함수
%cd /content/drive/MyDrive/project3

# Add the paths to the modules to the Python path
import sys
sys.path.append(os.path.abspath('./code/image_function'))

from image_load import resize_image, load_image, load_and_resize_images
from vram_reset import reset_vram

/content/drive/.shortcut-targets-by-id/1gdU-KuXDUU7v2o8X7TdV0JgGiz603mCc/project3


In [13]:
food_middle_list = [] # 중분류
food_path_list = [] # 중분류 경로
datadir = '/content/drive/MyDrive/project3/data/traindata/kfood/'
for food_middle in sorted(os.listdir(datadir)):
    middle_path = os.path.join(datadir,food_middle)
    food_middle_list.append(food_middle)

    for food in os.listdir(middle_path):
      path = os.path.join(middle_path, food)
      food_path_list.append(path)

print(len(food_middle_list))
print(food_middle_list)
print(food_path_list)

13
['구이', '국', '기타', '김치', '나물', '떡', '만두', '면', '무침', '밥', '볶음', '쌈', '음청류']
['/content/drive/MyDrive/project3/data/traindata/kfood/구이/갈비구이', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/갈치구이', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/고등어구이', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/곱창구이', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/닭갈비', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/장어구이', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/조개구이', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/떡갈비', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/불고기', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/삼겹살', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/더덕구이', '/content/drive/MyDrive/project3/data/traindata/kfood/구이/조기구이', '/content/drive/MyDrive/project3/data

In [14]:
image_list, target_list, except_list = load_and_resize_images(food_path_list, food_middle_list)

Processing images: 100%|██████████| 13/13 [02:45<00:00, 12.75s/it]


In [21]:
flat_data = image_list.flatten()

In [ ]:
target = []
images = []   # In matrix format
flat_data = []  # In vector format

rootdir = '/content/drive/MyDrive/project3/data/traindata/kfood/'
for file in os.listdir(rootdir):
    datadir = os.path.join(rootdir, file)
    for category in os.listdir(datadir):
      categorypath = os.path.join(datadir,category)
      for idx, img in enumerate(os.listdir(categorypath)):
        if idx == 100:
          break
        try:
          img_matrix=imread(os.path.join(categorypath,img))
          img_resized=resize(img_matrix,(150,150,3))
          flat_data.append(img_resized.flatten())
          images.append(img_resized)
          target.append(file)
        except Exception as someReadingError:
          print(someReadingError)

In [25]:
df.head()

,0
0,15.0
1,26.0
2,48.0
3,20.0
4,43.0


In [23]:
import pandas as pd
df=pd.DataFrame(flat_data)
df['Target']=target_list
df

ValueError: Length of values (1298) does not match length of index (87615000)

In [ ]:
# Input and Output
x=df.iloc[:,0:67500].values
y=df.iloc[:,67500].values

### 3) Classification of images using KNN Classification Algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = KNeighborsClassifier(n_neighbors=16,metric='euclidean')
model.fit(x_train,y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=16)

In [ ]:
# Predicted output
y_pred = model.predict(x_test)
y_pred

array(['고사리나물', '고사리나물', '숙주나물', ..., '가지볶음', '고사리나물', '고사리나물'],
      dtype=object)

In [ ]:
# Actual output
y_test

array(['시금치나물', '미역줄기볶음', '애호박볶음', ..., '고사리나물', '시금치나물', '고사리나물'],
      dtype=object)

In [ ]:
# Evaluation of the model
from sklearn.metrics import accuracy_score
print("The accuracy of the model is ",accuracy_score(y_pred,y_test)*100,"%")

The accuracy of the model is  30.266666666666666 %


### 4) Confusion Matrix and Classification report

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_pred,y_test)
cm

array([[ 20,  16,   6,   1,  15,   2],
       [ 33,  75,  31,   1,  61,   4],
       [ 45,  31,  52,   9,  64,  16],
       [137, 112, 131, 238,  73, 195],
       [  2,   8,  12,   0,  48,   0],
       [  8,   7,   6,   4,  16,  21]])

In [ ]:
# Classification report
from sklearn import metrics
metrics.classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n        가지볶음       0.33      0.08      0.13       245\n       고사리나물       0.37      0.30      0.33       249\n      미역줄기볶음       0.24      0.22      0.23       238\n        숙주나물       0.27      0.94      0.42       253\n       시금치나물       0.69      0.17      0.28       277\n       애호박볶음       0.34      0.09      0.14       238\n\n    accuracy                           0.30      1500\n   macro avg       0.37      0.30      0.25      1500\nweighted avg       0.38      0.30      0.26      1500\n'